## 电脑必选Edge或者Chrome
## Please use Edge or Chrome! Or you couldnt download the file from the Kaggle!!
## 目的是下载kaggle的blog二进制文件。很少有浏览器能支持大规模的blob传输。
## 只有上述浏览器才能成功下载保存的zip文件

# Before training, Do not forget turn on the settings of "INTERNET"! And Do not forget choose GPU as your accelerator!(Tesla T2 recommended)
# 在训练前注意打开internet,将Accelerator改成任意一个GPU(建议T4)
> You may need verify your phone number to be accessible to GPU if you are new Kaggle user.
对于新手玩家需要验证一下手机号才能使用GPU。

# 注意，目前只能使用训练webui，无法使用uvr5,和推理的webui。
### 解决方法:自己fork一下官方仓库，修改./GPT_SoVITS/inference_webui.py的gradio的share属性为true。同理uvr5也如此，将第一步的原仓库改成你的仓库地址。即可进行推理。输出gradio的public link

$ C_{n}^{m} = \frac{n!}{m!(n-m)!} $
$ \text{最诚挚的祝福} $

## 所有提示信息都写在每个代码框的注释里。

# 1.必要:安装环境(可替换你的fork地址)
# 1. Important: Install Dep(Always run it first whatever happened)

In [ ]:
#准备环境,不管以前保没保存环境都必做.
!git clone --depth=1 https://github.com/RVC-Boss/GPT-SoVITS.git
%cd GPT-SoVITS
!apt-get update && apt-get install -y --no-install-recommends tzdata ffmpeg libsox-dev parallel aria2 git git-lfs && git lfs install
!pip install -r requirements.txt

# 2.下载预训练模型
# 2. Download pre-trained models for following traning.
### You dont have to run if you have saved your env(check it out through settings, see if it has 10G or more data )

In [ ]:
#注意! 如果以前保存过环境就没必要进行这个代码框了!直接去下一个代码框!!!!
#具体点开你右上角菜单，看是否有10G以上的数据存放在里面。或者说在启动的时候加载很慢，提示
# Loading data之类的。
!mkdir -p /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models
!mkdir -p /kaggle/working/GPT-SoVITS/tools/asr/models
!mkdir -p /kaggle/working/GPT-SoVITS/tools/uvr5
%cd /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models
!git clone https://huggingface.co/lj1995/GPT-SoVITS
%cd /kaggle/working/GPT-SoVITS/tools/asr/models
!git clone https://www.modelscope.cn/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.git
!git clone https://www.modelscope.cn/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.git
!git clone https://www.modelscope.cn/damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch.git

# 这里注释掉uvr5模型是因为你用不上它。也是用不了。
# 除了开头的方式,如果你有更好的内网穿透工具并且知道如何引用，欢迎更新version。
# 来穿透uvr5 webui端口可以一试。

#%cd /kaggle/working/GPT-SoVITS/tools/uvr5
#!git clone https://huggingface.co/Delik/uvr5_weights
#!git config core.sparseCheckout true

!mv /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models/GPT-SoVITS/* /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models/

# Webui 启动!

In [ ]:
#启动webui.py
# 通过npm插件启动单一端口的内网穿透。不能多开。
#  也就是只能训练，不能用来云端推理和进行uvr5的降噪

%cd /kaggle/working/GPT-SoVITS/
!npm install -g localtunnel
import subprocess
import threading
import time
import socket
import urllib.request
def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock= socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
        from colorama import Fore, Style
    print (Fore.GREEN + "\nIP: ", Fore. RED, urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"), "\n", Style. RESET_ALL)
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')
threading.Thread (target=iframe_thread, daemon=True, args=(9874,)).start()
!pip install --upgrade gradio
!python  webui.py

***执行下一步之前必须要停止webui进程才能继续!***

***When it comes to pack your weights,stop every running cell first.***

In [ ]:
#打包权重文件到/kaggle/working/GPT-SoVITS文件夹内，点击刷新按钮即可看到大包邮的内容.
%cd /kaggle/working/GPT-SoVITS/
!zip AnyName_GPT_packed.zip /kaggle/working/GPT-SoVITS/SoVITS_weights/* /kaggle/working/GPT-SoVITS/GPT_weights/*

In [ ]:
#清理GPU显存
#当你用到它的时候，大概率没什么用。肯定是webui填写的时候GPU卡号多写了一组，例如0-1,0-1。正确的应该是
# 0-1.也有可能batch size太高爆显存了，从7拉低到5即可完美解决。
!nvidia-smi
from numba import cuda
import torch
cuda.select_device(0)
cuda.close()
cuda.select_device(1)
cuda.close()
torch.cuda.empty_cache()

## 保持neat: 删除所有音频用来下一次训练
## Keep tidy and neat! the cell below will delete akl cache data for trainibg next character.

In [ ]:
# 完全重置，用于新的训练。
%cd /kaggle/working/GPT-SoVITS
#这部分必须删除，否则会对新模型训练产生不可逆转的影响。
!rm -rf ./output/slicer_opt/*.wav #删除你的切片文件
!rm -rf ./output/denoise_opt/* #删除你降噪后的文件
!rm -rf ./log/* # 删除log会清空训练过程中产生的数据。但不会清空结果生成的权重。
# 如果你不是想拿它继续训练，就可以删掉它。

# !rm -rf ./*.zip

#注意，为了节省空间，建议将打包的zip文件执行删除!
#请在执行之前确保你已经下载完权重文件，为避免误操作，已经注释这个功能
#若不想删除，保持注释上面这一行即可。但是随着训练任务的增多
#会不知不觉地占用你的空间
!rm -rf ./output/asr_opt/* #删除生成的文本文件
!rm -rf ./input/* #删除你的样本未处理文件

#为避免误操作，以下为注释内容。
# rm -rf ./GPT_SoVITS_weight/*
# rm -rf ./SoVITS_weight/*